In [1]:
import tensorflow as tf
import polars as pl
import utils
utils.mount_src()

DATA_PATH = "E:/30-39 Estudos/36 Mestrado/36.03 Deep learning para computação visual/Projeto/flickr_scraper/images"
METADATA = DATA_PATH + "/metadata.csv"
EPOCHS = 1

In [ ]:
"""
import pathlib
import polars as pl
import filetype
import tensorflow as tf


class DataLoader:
    data_path: pathlib.Path
    df: pl.DataFrame


    def __init__(self, data_path: str):
        self.data_path = pathlib.Path(data_path)
        self.df = pl.read_csv(self.data_path / "metadata.csv")
    

    def dataset(self, target_size=(224, 224), batch_size=32) -> tf.data.Dataset:
        # Clean-up dataset
        self.df = self.__filter_out_gifs(self.df)
        self.df = self.__filter_out_non_existent_files(self.df)

        paths: pl.Series = str(self.data_path) + "/" + self.df["year"].cast(pl.String) 
        paths += "/" + self.df["id"].cast(pl.String) + ".jpg"
        paths = paths.to_numpy()
        years = self.df["year"].to_numpy()
        lats = self.df["latitude"].to_numpy()
        lons = self.df["longitude"].to_numpy()

        dataset = tf.data.Dataset.from_tensor_slices((paths, years, lats, lons))
        dataset = dataset.map(
            lambda path, year, lat, lon: DataLoader.load_image_and_labels(path, year, lat, lon, image_shape=target_size)
        )

        dataset = dataset.shuffle(buffer_size=len(paths)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset


    @staticmethod
    def load_image_and_labels(path: str, year: int, lat: float, lon: float, image_shape=(224, 224)):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, image_shape) / 255.0
        return img, {
            "year": year,
            "lat": lat,
            "lon": lon
        }


    def __filter_out_gifs(self, df: pl.DataFrame) -> pl.DataFrame:
        def is_gif(filepath: str) -> bool:
            guess = filetype.guess(filepath)
            return guess.mime == "image/gif"
        
        original_columns = df.columns
        df = df.with_columns(
            (
                str(self.data_path) + "/" + pl.col("year").cast(pl.String) + "/" + pl.col("id").cast(pl.String) + ".jpg"
            ).alias("filepath")
        )
        df = df.with_columns(
            (
                df["filepath"].map_elements(is_gif, return_dtype=pl.Boolean)
            ).alias("is_gif")
        )
        df = df.filter(~pl.col("is_gif"))
        return df.select(original_columns)
    

    def __filter_out_non_existent_files(self, df: pl.DataFrame) -> pl.DataFrame:
        def file_exists(filepath: str) -> bool:
            file = pathlib.Path(filepath)
            return file.is_file()
        
        original_columns = df.columns
        df = df.with_columns(
            (
                str(self.data_path) + "/" + pl.col("year").cast(pl.String) + "/" + pl.col("id").cast(pl.String) + ".jpg"
            ).alias("filepath")
        )
        df = df.with_columns(
            (
                df["filepath"].map_elements(file_exists, return_dtype=pl.Boolean)
            ).alias("file_exists")
        )
        df = df.filter(pl.col("file_exists"))
        return df.select(original_columns)
"""

In [ ]:
"""
data_loader = DataLoader(DATA_PATH)
print("Original df length: " + str(len(data_loader.df)))
df = data_loader.cleanup_df()
data_loader.df
"""

In [ ]:
"""
paths = DATA_PATH + "/" + data_loader.df["year"].cast(pl.String) + "/" + data_loader.df["id"].cast(pl.String) + ".jpg"
paths = paths.to_numpy()
years = data_loader.df["year"].to_numpy()
lats = data_loader.df["latitude"].to_numpy()
lons = data_loader.df["longitude"].to_numpy()
"""

In [ ]:
"""
def load_image_and_label(path: str, year: int, lat: float, lon: float, image_shape=(224, 224)):
    try:
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, image_shape) / 255.0
    except tf.errors.InvalidArgumentError as e:
        tf.print(f"Error on path {path}")
        raise e
    return img, {
        "year": year,
        "lat": lat,
        "lon": lon
    }
"""

In [ ]:
"""
#tf.config.run_functions_eagerly(True)
#tf.data.experimental.enable_debug_mode()

dataset = tf.data.Dataset.from_tensor_slices((paths, years, lats, lons))
dataset = dataset.map(lambda path, year, lat, lon: load_image_and_label(path, year, lat, lon))

batch_size = 32
dataset = dataset.shuffle(buffer_size=len(paths)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
"""

In [7]:
def create_model():
    input_img = tf.keras.Input(shape=(224, 224, 3), name="image")
    x = tf.keras.layers.Conv2D(16, (3, 3), activation="relu")(input_img)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(32, activation="relu")(x)

    output_year = tf.keras.layers.Dense(1, name="year")(x)
    output_lat = tf.keras.layers.Dense(1, name="lat")(x)
    output_lon = tf.keras.layers.Dense(1, name="lon")(x)

    return tf.keras.Model(inputs=input_img, outputs={"year": output_year, "lat": output_lat, "lon": output_lon})


In [8]:
model = create_model()
#model.build(input_shape=(None, 224, 224, 3))
model.compile(
    optimizer="adam",
    loss={"year": "mse", "lat": "mse", "lon": "mse"},
    metrics={"year": "mae", "lat": "mae", "lon": "mae"}
)

In [ ]:
from data_loader import DataLoader
data_loader = DataLoader(DATA_PATH)
dataset = data_loader.dataset()

In [ ]:
history = model.fit(dataset, epochs=EPOCHS)

In [ ]:
history.history

In [ ]:
"""
import matplotlib.pyplot as plt

epochs = [i for i in range(1, 6)]

plt.plot(epochs, history.history["lon_mae"])
"""

In [ ]:
"""
from tensorflow.keras.utils import img_to_array, load_img

example = DATA_PATH + "/1929/" + "2123423918.jpg"
img = load_img(example)
img
"""

In [ ]:
"""
metadata_for_year = pl.read_csv(DATA_PATH + "/1929/metadata.csv")
labels = metadata_for_year.filter(pl.col("id") == 2123423918)
labels
"""

In [ ]:
"""
def load_and_preprocess_image(path, image_shape=(224, 224)):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, image_shape) / 255.0
    return img

img = load_and_preprocess_image(example)
img_batch = tf.expand_dims(img, axis=0)
model.predict(img_batch)
"""